In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
#print(os.listdir("/kaggle/input/test"))
!pip install efficientnet_pytorch
!pip install torchsummary
from efficientnet_pytorch import EfficientNet
import torchvision
import torch
from torch import nn
import torch.nn.functional as F
import torchvision.models as models
from torchsummary import summary
import torch.optim as optim
import copy
import os
import torch
from tqdm.autonotebook import tqdm
from torch.optim.lr_scheduler import _LRScheduler
import matplotlib.pyplot as plt
from PIL import Image
from torch.utils.data import Dataset
# Any results you write to the current directory are saved as output.

/usr/local/lib/python3.6/dist-packages/tqdm/autonotebook/__init__.py:14: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  " (e.g. in jupyter console)", TqdmExperimentalWarning)


In [2]:
!pip install -i https://test.pypi.org/simple/ supportlib
import supportlib.gettingdata as getdata
getdata.kaggle()

Looking in indexes: https://test.pypi.org/simple/


Saving kaggle.json to kaggle.json


In [3]:
!kaggle competitions download -c aerial-cactus-identification

  0% 0.00/667k [00:00<?, ?B/s]
100% 667k/667k [00:00<00:00, 46.5MB/s]
  0% 0.00/160k [00:00<?, ?B/s]
100% 160k/160k [00:00<00:00, 47.5MB/s]
  0% 0.00/4.20M [00:00<?, ?B/s]
100% 4.20M/4.20M [00:00<00:00, 38.5MB/s]
 47% 9.00M/19.2M [00:00<00:00, 32.0MB/s]
100% 19.2M/19.2M [00:00<00:00, 55.2MB/s]


In [0]:
getdata.zipextract('/content/test.zip')
getdata.zipextract('/content/train.zip')

In [5]:
train_csv = pd.read_csv('/content/train.csv')
train_csv.head(10)

,id,has_cactus
0,0004be2cfeaba1c0361d39e2b000257b.jpg,1
1,000c8a36845c0208e833c79c1bffedd1.jpg,1
2,000d1e9a533f62e55c289303b072733d.jpg,1
3,0011485b40695e9138e92d0b3fb55128.jpg,1
4,0014d7a11e90b62848904c1418fc8cf2.jpg,1
5,0017c3c18ddd57a2ea6f9848c79d83d2.jpg,1
6,002134abf28af54575c18741b89dd2a4.jpg,0
7,0024320f43bdd490562246435af4f90b.jpg,0
8,002930423b9840e67e5a54afd4768a1e.jpg,1
9,00351838ebf6dff6e53056e00a1e307c.jpg,1


In [0]:
classes = train_csv['has_cactus'].unique()
encoder = {0:'no cactus',1:'has cactus'}

In [0]:
from sklearn.model_selection import train_test_split
train_df,val_df = train_test_split(train_csv,test_size = 0.1)
val_df = val_df.reset_index()
val_df = val_df.drop(['index'],axis = 1)
train_df = train_df.reset_index()
train_df = train_df.drop(['index'],axis = 1)

In [0]:
class cactus_dataset(Dataset):
  def __init__(self,image_dir,train_csv,transform = None):
    self.img_dir = image_dir
    self.transform = transform
    self.id = train_csv.iloc[:,0]
    self.classes =  train_csv.iloc[:,1]
  def __len__(self):
    return len(self.id)
  def __getitem__(self,idx):
    img_name = os.path.join(self.img_dir, self.id[idx])
    image = cv2.imread(img_name)
    if self.transform:
        image = self.transform(image)
    label = self.classes[idx]
    return image,label


In [0]:
batch_size = 256
import cv2
from torchvision import transforms
from torch.utils.data import DataLoader


In [0]:
train_transforms = transforms.Compose([
                                        transforms.ToPILImage(),
                                    
                                        #transforms.RandomResizedCrop(224),                                    
                                        transforms.RandomHorizontalFlip(),
                                        transforms.RandomVerticalFlip(),
                                        
    transforms.ToTensor(),
                                        transforms.Normalize([0.485, 0.456, 0.406], 
                                                            [0.229, 0.224, 0.225])])
test_transforms = transforms.Compose([
                                        transforms.ToPILImage(),
                                        #transforms.Resize(256),
                                       #   transforms.CenterCrop(224),
                                        transforms.ToTensor(),
                                        transforms.Normalize([0.485, 0.456, 0.406], 
                                                            [0.229, 0.224, 0.225])])

#inverse normalization for image plot
train_data = cactus_dataset('/content/train',train_csv,transform = train_transforms)

In [0]:
#val_data = cactus_dataset('/content/train',val_df,transform = test_transforms)
train_loader = DataLoader(train_data, batch_size=batch_size,
                        shuffle=True, num_workers=0)

#val_loader = DataLoader(val_data, batch_size=batch_size,shuffle=True, num_workers=0)
dataloaders = {'train':train_loader}

In [0]:
model = models.densenet161()
for child in model.children():
  for layer in child.modules():
    layer.requires_grad = False
    if(isinstance(layer,torch.nn.modules.batchnorm.BatchNorm2d)):
      layer.requires_grad = True

In [0]:
from efficientnet_pytorch import EfficientNet
import torchvision
import torch
from torch import nn
import torch.nn.functional as F
import torchvision.models as models
from torchsummary import summary
import torch.optim as optim
import copy
import os
import torch
from tqdm.autonotebook import tqdm

import matplotlib.pyplot as plt

class classifie(nn.Module):
    def __init__(self):
        super(classifie, self).__init__()
        model = models.densenet161(pretrained = True)
        model = model.features
        for child in model.children():
          for layer in child.modules():
            layer.requires_grad = False
            if(isinstance(layer,torch.nn.modules.batchnorm.BatchNorm2d)):
              layer.requires_grad = True
        self.model = model
        self.linear = nn.Linear(4416, 512)
        self.bn = nn.BatchNorm1d(512)
        self.dropout = nn.Dropout(0.5)
        self.elu = nn.ELU()
        self.out = nn.Linear(512, 2)
        self.bn1 = nn.BatchNorm1d(4416)
        self.dropout2 = nn.Dropout(0.2)
    def forward(self, x):
        out = self.model(x)
        avg_pool = nn.functional.adaptive_avg_pool2d(out, output_size = 1)
        max_pool = nn.functional.adaptive_max_pool2d(out, output_size = 1)
        out = torch.cat((avg_pool,max_pool),1)
        batch = out.shape[0]
        out = out.view(batch, -1)
        conc = self.linear(self.dropout2(self.bn1(out)))
        conc = self.elu(conc)
        conc = self.bn(conc)
        conc = self.dropout(conc)
        res = self.out(conc)
        return out

In [42]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
classifier = classifie().to(device)

Downloading: "https://download.pytorch.org/models/densenet161-8d451a50.pth" to /root/.cache/torch/checkpoints/densenet161-8d451a50.pth
100%|██████████| 115730790/115730790 [00:01<00:00, 76342895.35it/s]


In [0]:
import torch.optim as optim
import matplotlib.pyplot as plt
import random
from torch.autograd import Variable
import numpy as np
import torch
from torch import nn
import sys
def train(model,dataloaders,device,num_epochs,lr,batch_size,patience):
    phase1 = dataloaders.keys()
    losses = list()
    criterion = nn.CrossEntropyLoss()
    acc = list()
    for epoch in range(num_epochs):
        print('Epoch:',epoch)
        optimizer = optim.Adam(model.parameters(), lr=lr,weight_decay = 1e-6)
        lr = lr*0.9
        for phase in phase1:
            epoch_metrics = {"loss": [], "acc": []}
            if phase == ' train':
                model.train()
            else:
                model.eval()
            for  batch_idx, (data, target) in enumerate(dataloaders[phase]):
                data, target = Variable(data), Variable(target)
                data = data.type(torch.FloatTensor).to(device)
                target = target.type(torch.LongTensor).to(device)

                optimizer.zero_grad()
                output = model(data)
                loss = criterion(output, target)
                target = target.type(torch.LongTensor).to(device)

                acc = 100 * (output.detach().argmax(1) == target).cpu().numpy().mean()
                epoch_metrics["loss"].append(loss.item())
                epoch_metrics["acc"].append(acc)
                if(phase =='train'):
                    loss.backward()
                    optimizer.step()
                sys.stdout.write(
                "\r[Epoch %d/%d] [Batch %d/%d] [Loss: %f (%f), Acc: %.2f%% (%.2f%%)]"
                % (
                    epoch,
                    num_epochs,
                    batch_idx,
                    len(dataloaders[phase]),
                    loss.item(),
                    np.mean(epoch_metrics["loss"]),
                    acc,
                    np.mean(epoch_metrics["acc"]),
                    )
                )
               
            epoch_acc = np.mean(epoch_metrics["acc"])
            epoch_loss = np.mean(epoch_metrics["loss"])
        print('')  
        print('{} Accuracy: {}'.format(phase,epoch_acc.item()))
    return losses,acc

def train_model(model,dataloaders,encoder,lr_scheduler = None,inv_normalize = None,num_epochs=10,lr=0.0001,batch_size=8,patience = None,classes = None):
    dataloader_train = {}
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    losses = list()
    accuracy = list()
    key = dataloaders.keys()
    perform_test = False
    for phase in key:
        if(phase == 'test'):
            perform_test = True
        else:
            dataloader_train.update([(phase,dataloaders[phase])])
    losses,accuracy = train(model,dataloader_train,device,num_epochs,lr,batch_size,patience)

In [0]:
import cv2
lr = 0.001
train_model(classifier,dataloaders,encoder,inv_normalize = None,num_epochs=8,lr = lr,batch_size = batch_size,patience = None,classes = classes)

Epoch: 0
[Epoch 0/8] [Batch 68/69] [Loss: 1.239883 (4.685680), Acc: 76.09% (39.04%)]
train Accuracy: 39.04428560176434
Epoch: 1
[Epoch 1/8] [Batch 68/69] [Loss: 1.531342 (5.491149), Acc: 72.83% (67.87%)]
train Accuracy: 67.874970463138
Epoch: 2
[Epoch 2/8] [Batch 68/69] [Loss: 0.882876 (2.020590), Acc: 93.48% (77.23%)]
train Accuracy: 77.22658711405167
Epoch: 3
[Epoch 3/8] [Batch 68/69] [Loss: 0.792794 (0.863688), Acc: 97.83% (93.59%)]
train Accuracy: 93.58828568052931
Epoch: 4
[Epoch 4/8] [Batch 68/69] [Loss: 0.711749 (0.772140), Acc: 100.00% (97.31%)]
train Accuracy: 97.31091485507247
Epoch: 5
[Epoch 5/8] [Batch 56/69] [Loss: 0.732071 (0.746180), Acc: 98.83% (98.07%)]

In [0]:
for param in classifier.parameters():
  param.requires_grad = True

In [295]:
lr = 0.0005
train_model(classifier,dataloaders,encoder,inv_normalize = None,num_epochs=6,lr = lr,batch_size = batch_size,patience = None,classes = classes)

Epoch: 0
[Epoch 0/6] [Batch 68/69] [Loss: 0.776880 (0.719009), Acc: 96.74% (99.17%)]
train Accuracy: 99.17149102079395
Epoch: 1
[Epoch 1/6] [Batch 68/69] [Loss: 0.694665 (0.713344), Acc: 100.00% (99.28%)]
train Accuracy: 99.28102355072464
Epoch: 2
[Epoch 2/6] [Batch 68/69] [Loss: 0.708595 (0.709714), Acc: 98.91% (99.48%)]
train Accuracy: 99.48039736925016
Epoch: 3
[Epoch 3/6] [Batch 68/69] [Loss: 0.727408 (0.707603), Acc: 98.91% (99.47%)]
train Accuracy: 99.47473613736611
Epoch: 4
[Epoch 4/6] [Batch 68/69] [Loss: 0.703082 (0.706080), Acc: 98.91% (99.55%)]
train Accuracy: 99.5539933837429
Epoch: 5
[Epoch 5/6] [Batch 68/69] [Loss: 0.791379 (0.704066), Acc: 96.74% (99.65%)]
train Accuracy: 99.64703449905483


In [0]:
lr = 0.0001
train_model(classifier,dataloaders,encoder,inv_normalize = None,num_epochs=5,lr = lr,batch_size = batch_size,patience = None,classes = classes)

Epoch: 0
[Epoch 0/5] [Batch 68/69] [Loss: 0.751846 (0.700689), Acc: 97.83% (99.73%)]
train Accuracy: 99.72506104284814
Epoch: 1
[Epoch 1/5] [Batch 68/69] [Loss: 0.696269 (0.699244), Acc: 100.00% (99.81%)]
train Accuracy: 99.80751811594203
Epoch: 2
[Epoch 2/5] [Batch 68/69] [Loss: 0.693611 (0.698691), Acc: 100.00% (99.83%)]
train Accuracy: 99.83016304347827
Epoch: 3
[Epoch 3/5] [Batch 61/69] [Loss: 0.703136 (0.698566), Acc: 99.61% (99.83%)]

In [128]:
torch.save(classifier.state_dict(),'/content/cactusdense121.h5')
file1 = drive.CreateFile({"mimeType": "h5/h5"})
file1.SetContentFile("/content/cactusdense121.h5")
file1.Upload()

InvalidConfigError: ignored

In [0]:
file1 = drive.CreateFile({"mimeType": "h5/h5"})
file1.SetContentFile("/content/cactuseffnetb3_2.h5")
file1.Upload()

In [0]:
torch.save(classifier.state_dict(),'/content/cactusdensenet201_1.h5')


In [0]:
file1 = drive.CreateFile({"mimeType": "h5/h5"})
file1.SetContentFile("/content/cactusdensenet201_1.h5")
file1.Upload()

In [130]:
!pip install pydrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
file1 = drive.CreateFile({"mimeType": "h5/h5"})
file1.SetContentFile("/content/cactuseffnetb3_1.h5")
file1.Upload()

In [36]:
from google_drive_downloader import GoogleDriveDownloader as gdd

gdd.download_file_from_google_drive(file_id='1d78MJhBvBnfqjNDPy-kmNy7RazyUPsQ4',
                                    dest_path='/content/weights1.h5')

Unzipping...

/usr/local/lib/python3.6/dist-packages/google_drive_downloader/google_drive_downloader.py:78: UserWarning: Ignoring `unzip` since "1d78MJhBvBnfqjNDPy-kmNy7RazyUPsQ4" does not look like a valid zip file
  warnings.warn('Ignoring `unzip` since "{}" does not look like a valid zip file'.format(file_id))


In [0]:
class cactus_dataset_test(Dataset):
  def __init__(self,image_dir,transform = None):
    self.img_dir = image_dir
    self.transform = transform
    self.id = os.listdir(image_dir)
  def __len__(self):
    return len(self.id)
  def __getitem__(self,idx):
    img_name = os.path.join(self.img_dir, self.id[idx])
    image = cv2.imread(img_name)
    if self.transform:
      image = self.transform(image)
    return (self.id[idx],image)

In [0]:
test1 = cactus_dataset_test('/kaggle/input/test/test',test_transforms)

In [0]:
test_loader = DataLoader(test1, batch_size =16, shuffle = True)

In [0]:
def test(model,dataloader,device,batch_size):
    running_corrects = 0
    running_loss=0
    pred = []
    id = list()
    sm = nn.Softmax(dim = 1)
    criterion = nn.CrossEntropyLoss()
    for batch_idx, (id_1,data) in enumerate(dataloader):
        data = Variable(data)
        data = data.type(torch.FloatTensor).to(device)
        model.eval()
        output = model(data)
        output = sm(output)
        _, preds = torch.max(output, 1)
        preds = preds.cpu().numpy()
        preds = np.reshape(preds,(len(preds),1))
        
        for i in range(len(preds)):
            pred.append(preds[i])
            id.append(id_1[i])
    return id,pred

In [0]:
import torch.optim as optim
import matplotlib.pyplot as plt
from torch.autograd import Variable
import numpy as np
import torch
from torch import nn
id,pred = test(classifier,test_loader,'cuda',32)

In [0]:
a = list()
for i in range(len(pred)):
    a.append(pred[i][0])

In [0]:
a = np.asarray(a)

In [0]:
a = np.reshape(a,(-1,1))

In [0]:
b = np.asarray(id)

In [0]:
b = np.reshape(b,(-1,1))

In [0]:
sub = np.concatenate((b,a),axis = 1)

In [0]:
sub_df = pd.DataFrame(sub)

In [0]:
sub_df.columns = ['id','has_cactus']

In [0]:
sub_df.head(10)

In [0]:
sub_df.to_csv("/kaggle/working/submission.csv", index=False)